![(book cover)](https://covers.oreillystatic.com/images/0636920167433/cat.gif "(book cover)")
# "Programming Quantum Computers" by O'Reilly Media -  [book info](http://shop.oreilly.com/product/0636920167433.do)  - [all code samples](https://oreilly-qc.github.io)

## Code samples for Chapter 7
These code samples were written by Mariia Mykhailova.

### Example 7-1: Using QFT to distinguish between three states

In [1]:
// Example 7-1: Using QFT to distinguish between three states

open Microsoft.Quantum.Arithmetic;
open Microsoft.Quantum.Diagnostics;

operation PrepareState (register : Qubit[], state : String) : Unit {
    ApplyToEach(H, register);
    if (state == "A") {
        Z(register[0]);
    } elif (state == "B") {
        S(register[0]);
        Z(register[1]);
    } else {
        T(register[0]);
        S(register[1]);
        Z(register[2]);
    }
}

// An operation that is given a register in one of the states A, B or C
// and returns the letter of the state it recognized
operation ClassifyState (register : Qubit[]) : String {
    // Apply QFT
    // (Q# library implementation of QFT is inverse compared to the one used in the book,
    // so this code uses Adjoint QFTLE to match the desired effect).
    Adjoint QFTLE(LittleEndian(register));
    
    // Uncomment this line to see the amplitudes after QFT
    // DumpMachine();
    
    // Measure the register and interpret the results
    let frequency = MeasureInteger(LittleEndian(register));
    return frequency == 8 ? "A" | (frequency == 4 ? "B" | "C");
}

operation PrepareAndClassifyStates () : Unit {
    using (register = Qubit[4]) {
        for (inputState in ["A", "B", "C"]) {
            // Prepare the register in the given state
            PrepareState(register, inputState);
            
            // Uncomment this line to see the amplitudes of the prepared state
            // DumpMachine();
            
            // Classify the state
            let classifiedState = ClassifyState(register);
            Message($"State {inputState} recognized as {classifiedState}");
            
            // Make sure to reset qubits to the |0❭ state before reusing/releasing them
            ResetAll(register);
        }
    }
}

ClassifyState, PrepareAndClassifyStates, PrepareState

In [2]:
%simulate PrepareAndClassifyStates

State A recognized as A
State B recognized as B
State C recognized as C


()

### Example 7-2: QFT of simple QPU register signal

In [3]:
// Example 7-2: QFT of simple QPU register signal

open Microsoft.Quantum.Arithmetic;
open Microsoft.Quantum.Diagnostics;

operation SimpleStateQFT () : Unit {
    using (register = Qubit[4]) {
        // use PrepareState operation from previous sample to prepare the state
        PrepareState(register, "C");
        
        // Observe the amplitudes of the prepared state
        DumpMachine();
        
        // Apply QFT
        Adjoint QFTLE(LittleEndian(register));
        
        // Observe the amplitudes of the state after QFT transformation
        DumpMachine();
        
        // Make sure to reset qubits to the |0❭ state before releasing them
        ResetAll(register);
    }
}

SimpleStateQFT

In [4]:
%simulate SimpleStateQFT

# wave function for qubits with ids (least to most significant): 0;1;2;3
∣ 0❭:	 0.250000 +  0.000000 i	 == 	**                   [ 0.062500 ]     --- [  0.00000 rad ]
∣ 1❭:	 0.176777 +  0.176777 i	 == 	**                   [ 0.062500 ]      /  [  0.78540 rad ]
∣ 2❭:	 0.000000 +  0.250000 i	 == 	**                   [ 0.062500 ]    ↑    [  1.57080 rad ]
∣ 3❭:	-0.176777 +  0.176777 i	 == 	**                   [ 0.062500 ]  \      [  2.35619 rad ]
∣ 4❭:	-0.250000 +  0.000000 i	 == 	**                   [ 0.062500 ] ---     [  3.14159 rad ]
∣ 5❭:	-0.176777 + -0.176777 i	 == 	**                   [ 0.062500 ]  /      [ -2.35619 rad ]
∣ 6❭:	 0.000000 + -0.250000 i	 == 	**                   [ 0.062500 ]    ↓    [ -1.57080 rad ]
∣ 7❭:	 0.176777 + -0.176777 i	 == 	**                   [ 0.062500 ]      \  [ -0.78540 rad ]
∣ 8❭:	 0.250000 +  0.000000 i	 == 	**                   [ 0.062500 ]     --- [  0.00000 rad ]
∣ 9❭:	 0.176777 +  0.176777 i	 == 	**                   [ 0.062500 ]      /  [  0

()

### Example 7-3: Square-wave QFT

In [5]:
// Example 7-3: Square-wave QFT

open Microsoft.Quantum.Arithmetic;
open Microsoft.Quantum.Diagnostics;

operation SquareWaveQFT () : Unit {
    using (register = Qubit[4]) {
        let wavePeriod = 2;   // can range from 1 to 4
        // Prepare the state
        ApplyToEach(H, register);
        Z(register[wavePeriod - 1]);
        
        // Observe the amplitudes of the prepared state
        DumpMachine();
        
        // Apply QFT
        Adjoint QFTLE(LittleEndian(register));
        
        // Observe the amplitudes of the state after QFT transformation
        DumpMachine();
        
        // Make sure to reset qubits to the |0❭ state before releasing them
        ResetAll(register);
    }
}

SquareWaveQFT

In [6]:
%simulate SquareWaveQFT

# wave function for qubits with ids (least to most significant): 0;1;2;3
∣ 0❭:	 0.250000 +  0.000000 i	 == 	**                   [ 0.062500 ]     --- [  0.00000 rad ]
∣ 1❭:	 0.250000 +  0.000000 i	 == 	**                   [ 0.062500 ]     --- [  0.00000 rad ]
∣ 2❭:	-0.250000 +  0.000000 i	 == 	**                   [ 0.062500 ] ---     [  3.14159 rad ]
∣ 3❭:	-0.250000 +  0.000000 i	 == 	**                   [ 0.062500 ] ---     [  3.14159 rad ]
∣ 4❭:	 0.250000 +  0.000000 i	 == 	**                   [ 0.062500 ]     --- [  0.00000 rad ]
∣ 5❭:	 0.250000 +  0.000000 i	 == 	**                   [ 0.062500 ]     --- [  0.00000 rad ]
∣ 6❭:	-0.250000 +  0.000000 i	 == 	**                   [ 0.062500 ] ---     [  3.14159 rad ]
∣ 7❭:	-0.250000 +  0.000000 i	 == 	**                   [ 0.062500 ] ---     [  3.14159 rad ]
∣ 8❭:	 0.250000 +  0.000000 i	 == 	**                   [ 0.062500 ]     --- [  0.00000 rad ]
∣ 9❭:	 0.250000 +  0.000000 i	 == 	**                   [ 0.062500 ]     --- [  0

()

### Example 7-4: Square-wave QFT on a qubyte

In [7]:
// Example 7-4: Square-wave QFT on a qubyte

open Microsoft.Quantum.Arithmetic;
open Microsoft.Quantum.Diagnostics;

operation SquareWaveQFTQubyte () : Unit {
    using (register = Qubit[8]) {
        // Prepare the state
        ApplyToEach(H, register);
        Z(register[4]);
        
        // Observe the amplitudes of the prepared state
        DumpMachine();
        
        // Apply QFT
        Adjoint QFTLE(LittleEndian(register));
        
        // Observe the amplitudes of the state after QFT transformation
        DumpMachine();
        
        // Make sure to reset qubits to the |0❭ state before releasing them
        ResetAll(register);
    }
}

SquareWaveQFTQubyte

*(output of the next cell is not shown due to its size; run the cell to generate the output)*

In [ ]:
%simulate SquareWaveQFTQubyte

### Example 7-5: Converting frequency into state

In [9]:
// Example 7-5: Converting frequency into state

open Microsoft.Quantum.Arithmetic;
open Microsoft.Quantum.Diagnostics;

operation FrequencyToState () : Unit {
    using (register = Qubit[4]) {
        // Write frequency to register
        X(register[0]);
        X(register[1]);
        
        // Observe the amplitudes of the prepared state
        DumpMachine();
        
        // Apply QFT
        QFTLE(LittleEndian(register));
        
        // Observe the amplitudes of the state after QFT transformation
        DumpMachine();
        
        // Make sure to reset qubits to the |0❭ state before releasing them
        ResetAll(register);
    }
}

FrequencyToState

In [10]:
%simulate FrequencyToState

# wave function for qubits with ids (least to most significant): 0;1;2;3
∣ 0❭:	 0.000000 +  0.000000 i	 == 	                     [ 0.000000 ]                   
∣ 1❭:	 0.000000 +  0.000000 i	 == 	                     [ 0.000000 ]                   
∣ 2❭:	 0.000000 +  0.000000 i	 == 	                     [ 0.000000 ]                   
∣ 3❭:	 1.000000 +  0.000000 i	 == 	******************** [ 1.000000 ]     --- [  0.00000 rad ]
∣ 4❭:	 0.000000 +  0.000000 i	 == 	                     [ 0.000000 ]                   
∣ 5❭:	 0.000000 +  0.000000 i	 == 	                     [ 0.000000 ]                   
∣ 6❭:	 0.000000 +  0.000000 i	 == 	                     [ 0.000000 ]                   
∣ 7❭:	 0.000000 +  0.000000 i	 == 	                     [ 0.000000 ]                   
∣ 8❭:	 0.000000 +  0.000000 i	 == 	                     [ 0.000000 ]                   
∣ 9❭:	 0.000000 +  0.000000 i	 == 	                     [ 0.000000 ]                   
∣10❭:	 0.000000 +  0.000000 i	 == 	      

()

### Example 7-6: Prepare a state with varying ampitudes using QFT

In [11]:
// Example 7-6: Prepare a state with varying ampitudes using QFT

open Microsoft.Quantum.Arithmetic;
open Microsoft.Quantum.Arrays;
open Microsoft.Quantum.Diagnostics;

operation PrepareStateWithQFT () : Unit {
    using (register = Qubit[4]) {
        // Write the frequencies we want to register (a superposition of |1❭ and |15❭)
        H(register[0]);
        ApplyToEach(CNOT(Head(register), _), Rest(register));
        X(register[1]);
        CNOT(register[1], register[0]);
        X(register[1]);
        
        // Observe the amplitudes of the prepared state
        DumpMachine();
        
        // Apply QFT
        QFTLE(LittleEndian(register));
        
        // Observe the amplitudes of the state after QFT transformation
        DumpMachine();
        
        // Make sure to reset qubits to the |0❭ state before releasing them
        ResetAll(register);
    }
}

PrepareStateWithQFT

In [12]:
%simulate PrepareStateWithQFT

# wave function for qubits with ids (least to most significant): 0;1;2;3
∣ 0❭:	 0.000000 +  0.000000 i	 == 	                     [ 0.000000 ]                   
∣ 1❭:	 0.707107 +  0.000000 i	 == 	***********          [ 0.500000 ]     --- [  0.00000 rad ]
∣ 2❭:	 0.000000 +  0.000000 i	 == 	                     [ 0.000000 ]                   
∣ 3❭:	 0.000000 +  0.000000 i	 == 	                     [ 0.000000 ]                   
∣ 4❭:	 0.000000 +  0.000000 i	 == 	                     [ 0.000000 ]                   
∣ 5❭:	 0.000000 +  0.000000 i	 == 	                     [ 0.000000 ]                   
∣ 6❭:	 0.000000 +  0.000000 i	 == 	                     [ 0.000000 ]                   
∣ 7❭:	 0.000000 +  0.000000 i	 == 	                     [ 0.000000 ]                   
∣ 8❭:	 0.000000 +  0.000000 i	 == 	                     [ 0.000000 ]                   
∣ 9❭:	 0.000000 +  0.000000 i	 == 	                     [ 0.000000 ]                   
∣10❭:	 0.000000 +  0.000000 i	 == 	      

()

### Example 7-7: QFT frequency manipulation

In [13]:
// Example 7-7: QFT frequency manipulation

open Microsoft.Quantum.Arithmetic;
open Microsoft.Quantum.Diagnostics;

operation QFTFrequencyManipulation () : Unit {
    using (register = Qubit[4]) {
        // Write frequency to register
        X(register[1]);
        // Uncomment the line below to observe the amplitudes of the prepared state
        // DumpMachine();
        
        // Apply QFT to prepare our initial state
        let registerLE = LittleEndian(register);
        QFTLE(registerLE);
        // Observe the amplitudes of the initial state
        DumpMachine();
        
        // Move back to frequency space
        Adjoint QFTLE(registerLE);
        
        // Increase the frequency of the signal
        IncrementByInteger(1, registerLE);
        
        // Move back to signal space
        QFTLE(registerLE);
        // Observe the amplitudes of the resulting state
        DumpMachine();
        
        // Make sure to reset qubits to the |0❭ state before releasing them
        ResetAll(register);
    }
}

QFTFrequencyManipulation

In [14]:
%simulate QFTFrequencyManipulation

# wave function for qubits with ids (least to most significant): 0;1;2;3
∣ 0❭:	 0.250000 +  0.000000 i	 == 	**                   [ 0.062500 ]     --- [  0.00000 rad ]
∣ 1❭:	 0.176777 +  0.176777 i	 == 	**                   [ 0.062500 ]      /  [  0.78540 rad ]
∣ 2❭:	 0.000000 +  0.250000 i	 == 	**                   [ 0.062500 ]    ↑    [  1.57080 rad ]
∣ 3❭:	-0.176777 +  0.176777 i	 == 	**                   [ 0.062500 ]  \      [  2.35619 rad ]
∣ 4❭:	-0.250000 +  0.000000 i	 == 	**                   [ 0.062500 ] ---     [  3.14159 rad ]
∣ 5❭:	-0.176777 + -0.176777 i	 == 	**                   [ 0.062500 ]  /      [ -2.35619 rad ]
∣ 6❭:	 0.000000 + -0.250000 i	 == 	**                   [ 0.062500 ]    ↓    [ -1.57080 rad ]
∣ 7❭:	 0.176777 + -0.176777 i	 == 	**                   [ 0.062500 ]      \  [ -0.78540 rad ]
∣ 8❭:	 0.250000 +  0.000000 i	 == 	**                   [ 0.062500 ]     --- [  0.00000 rad ]
∣ 9❭:	 0.176777 +  0.176777 i	 == 	**                   [ 0.062500 ]      /  [  0

()

### Example 7-8: Rotating phases by different angles

In [15]:
// Example 7-8: Rotating phases by different angles

open Microsoft.Quantum.Convert;
open Microsoft.Quantum.Diagnostics;
open Microsoft.Quantum.Math;

operation RotatingPhases () : Unit {
    // Rotate k-th basis state by k*20 degrees
    let phi = 20.0 / 180.0 * PI();  // the rotation angle in radians
    
    using (register = Qubit[4]) {
        // Put the register in superposition of all basis states to see all results at once
        ApplyToEach(H, register);
        // DumpMachine();
        
        for (i in 0 .. 3) {
            R1(phi * IntAsDouble(1 <<< i), register[i]);
        }
        
        // Observe the amplitudes of the resulting state
        DumpMachine();
        
        // Make sure to reset qubits to the |0❭ state before releasing them
        ResetAll(register);
    }
}

RotatingPhases

In [16]:
%simulate RotatingPhases

# wave function for qubits with ids (least to most significant): 0;1;2;3
∣ 0❭:	 0.250000 +  0.000000 i	 == 	**                   [ 0.062500 ]     --- [  0.00000 rad ]
∣ 1❭:	 0.234923 +  0.085505 i	 == 	**                   [ 0.062500 ]      /- [  0.34907 rad ]
∣ 2❭:	 0.191511 +  0.160697 i	 == 	**                   [ 0.062500 ]      /  [  0.69813 rad ]
∣ 3❭:	 0.125000 +  0.216506 i	 == 	**                   [ 0.062500 ]     +/  [  1.04720 rad ]
∣ 4❭:	 0.043412 +  0.246202 i	 == 	**                   [ 0.062500 ]    ↑    [  1.39626 rad ]
∣ 5❭:	-0.043412 +  0.246202 i	 == 	**                   [ 0.062500 ]    ↑    [  1.74533 rad ]
∣ 6❭:	-0.125000 +  0.216506 i	 == 	**                   [ 0.062500 ]  \-     [  2.09440 rad ]
∣ 7❭:	-0.191511 +  0.160697 i	 == 	**                   [ 0.062500 ]  \      [  2.44346 rad ]
∣ 8❭:	-0.234923 +  0.085505 i	 == 	**                   [ 0.062500 ] +\      [  2.79253 rad ]
∣ 9❭:	-0.250000 +  0.000000 i	 == 	**                   [ 0.062500 ] ---     [  3

()